## Documentation

To read more about the ingest processor, checkout the docs [here](https://www.elastic.co/docs/reference/enrich-processor).



## Connect to ElasticSearch

In [ ]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

## Common ingest processors

Here’s a look at some frequently used ingest processors:

1. **Convert**: Changes the data type of a field.
2. **Rename**: Changes the name of a field.
3. **Set**: Assigns a specified value to a field.
4. **HTML Strip**: Strips HTML tags from a field's content.
5. **Lowercase**: Transforms the text in a field to lowercase.
6. **Uppercase**: Transforms the text in a field to uppercase.
7. **Trim**: Removes whitespace from the beginning and end of a field's value.
8. **Split**: Divides the field content into an array, using a comma `,` as the delimiter.
9. **Remove**: Deletes a field from the document.
10. **Append**: Adds a value to an array field.

### 1. Creating the document

We'll apply all these common processors to this document to demonstrate how each one works.

In [ ]:
document = {
    "price": "100.50",
    "old_name": "old_value",
    "description": "<p>This is a description with HTML.</p>",
    "username": "UserNAME",
    "category": "books",
    "title": "   Example Title with Whitespace   ",
    "tags": "tag1,tag2,tag3",
    "temporary_field": "This field should be removed"
}

### 2. Creating the pipeline

In [ ]:
pipeline_body = {
    "description": "Pipeline to demonstrate various ingest processors",
    "processors": [
        {
            "convert": {
                "field": "price",
                "type": "float",
                "ignore_missing": True
            }
        },
        {
            "rename": {
                "field": "old_name",
                "target_field": "new_name"
            }
        },
        {
            "set": {
                "field": "status",
                "value": "active"
            }
        },
        {
            "html_strip": {
                "field": "description"
            }
        },
        {
            "lowercase": {
                "field": "username"
            }
        },
        {
            "uppercase": {
                "field": "category"
            }
        },
        {
            "trim": {
                "field": "title"
            }
        },
        {
            "split": {
                "field": "tags",
                "separator": ","
            }
        },
        {
            "remove": {
                "field": "temporary_field"
            }
        },
        {
            "append": {
                "field": "tags",
                "value": ["new_tag"]
            }
        }
    ]
}

pipeline_id = "multi_steps_pipeline"

es.ingest.put_pipeline(id=pipeline_id, body=pipeline_body)

print(f"Pipeline '{pipeline_id}' created successfully!")

In [ ]:
from pprint import pprint

es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

response = es.index(index="my_index", document=document, pipeline=pipeline_id)
pprint(response.body)

In [ ]:
response = es.search(index='my_index')
hits = response.body['hits']['hits']

for hit in hits:
    pprint(hit['_source'])